# Tiny few shot classifer

Im folgenden wird ein few shot classifier für multilabels im [SetFit-Frame](https://huggingface.co/blog/setfit) Format erzeugt. Die multilabels-Option ist deshalb von Nöten, weil durchaus mehrere Future Skills in einem Kurs vermittelt werden können.

Gearbeitet wird mit `Python 3.12.3`, in einem eigens dafür erzeugten Poetry Enviorment (`Poetry Future Skills Enviorment`). Das Enviorment wird als Kernel händisch in Jupyter Notebook geladen. 

Weitere Pakete können dem Enviorment mit `poetry add <paketname>` hinzugefügt werden:

In [1]:
# !poetry add platformdirs
# !poetry add pyreadr

## Pakete laden

In einem weiteren Schritt werden die nötigen Pakete geladen. 

In [2]:
from datasets import load_dataset  # Zum Laden von Datensätzen aus der Hugging Face `datasets` Bibliothek
from sentence_transformers.losses import CosineSimilarityLoss  # Verlustfunktion basierend auf der Kosinusähnlichkeit, für Aufgaben wie Textähnlichkeit

from setfit import SetFitModel, SetFitTrainer  # Zum Laden des SetFit-Modells und Trainers, speziell für Few-Shot Learning mit Satztransformern
import pandas as pd  # Pandas für Datenmanipulation und -analyse
from sklearn.preprocessing import LabelEncoder  # Zum Encoden von Labels in numerische Werte für maschinelles Lernen
import os  # Für den Umgang mit Betriebssystemfunktionen (z.B. Dateipfade)
from datasets import Dataset, DatasetDict  # Zum Erstellen und Verwalten von Datensätzen in Hugging Face `datasets`-Format
from sklearn.model_selection import train_test_split  # Zum Aufteilen der Daten in Trainings- und Testsets

from transformers import TrainingArguments  # Zum Festlegen von Trainingsparametern für Modelle in der Transformers-Bibliothek

import torch  # Ermöglicht die GPU-Nutzung und effiziente Verarbeitung von Tensoren
import numpy as np  # Für numerische Berechnungen und Arbeiten mit Arrays

import re
import html

from unidecode import unidecode
from tqdm.auto import tqdm, trange
from time import sleep



c:\Users\Hueck\AppData\Local\pypoetry\Cache\virtualenvs\future-skill-classification--ys6-NbF-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Arbeitsverzeichnis bestimmen

Es wird weiterhin das Rootverzeichnis bestimmt um es später über relative Pfade Daten und Modelle laden zu können.

In [3]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification\Py


## Daten

### Daten laden

Von Yannic Hinrichs wurden per String-Match Trainingsdaten erzeugt, die um weitere Daten ergänzt wurden, die keine Future Skills enthalten. Die Daten wurden weiterhin durch Yannic Hinrichs händisch kontrolliert. 

Testdaten liegen zu jetzigen Zeitpunkt nicht vor.

In [4]:

# Laden der Trainingsdaten
df = pd.read_excel('data/train_data_franziska.xlsx')

# Im Fall von gemigemischten Daten: sentence-Spalte als String deklarieren
df['sentence'] = df['sentence'].astype(str)
 
# Falls wieder erwartend NAs in den Daten: Ersetze diese durch 0
df = df.fillna(0)

# Falls wieder erwartend infs in den Daten: Ersetze diese durch 0
df.replace([np.inf, -np.inf], 0, inplace=True)

# Falls wieder erwartend nötig: Die Werte aller numerischen Variablen werden ganze Zahlen gerundet.
df[df.select_dtypes(include='number').columns] = df.select_dtypes(include='number').astype('int64')

# Für die weitere Verarbeitung werden die Daten ins Hugging Face Dataset-Format transformiert. 

# # /////////////////////////////////////////////////
# # Hinweis: Das Dataset-Format der Hugging Face datasets-Bibliothek basiert auf Apache Arrow und ist speichereffizient, was schnelles Laden und Verarbeiten großer Datensätze ermöglicht. Es integriert sich nahtlos in Machine-Learning-Workflows und unterstützt einfache Transformationen und Vorverarbeitung durch Methoden wie .map(). Zudem ist es flexibel und kompatibel mit Pandas DataFrames sowie gängigen Frameworks wie PyTorch und TensorFlow.
# #//////////////////////////////////////////////////

dataset = Dataset.from_pandas(df)
print(dataset)
 

Dataset({
    features: ['sentence', 'Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing', 'Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten', 'Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 'Dialog- und Konfliktfähigkeit'],
    num_rows: 1577
})


### Daten verarbeiten

#### Feature Auswahl

Nun werden die Features des Classifiers ausgewählt. Diese werden von den Kurstitel und -beschreibungen (`sentence`) separiert. 

In [5]:
# Ziehe columnnames als Liste
features = dataset.column_names

# Definiere Columns, die nicht vom classifier berücksichtigt werden
to_remove = ['sentence','Digital Literacy', 'Digital Ethics', 'Digitale Kollaboration', 'Digital Learning', 'Agiles Arbeiten', 'Lösungsfähigkeit', 'Kreativität', 'Unternehmerisches Handeln & Eigeninitiative', 'Interkulturelle Kommunikation', 'Resilienz', 'Urteilsfähigkeit', 'Innovationskompetenz', 'Missionsorientierung', 'Veränderungskompetenz', 'Dialog- und Konfliktfähigkeit'] # Liste von Columns kann nach bedarf verlängert werden

# Schließe die oben definierten Variablen aus der Feature-Liste aus
filtered_features = [feature for feature in features if feature not in to_remove]

features = filtered_features

print(features)

['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing']


### One-Hot-Encoding

Für das Trainieren des Classifiers wird im folgenden ein *One-Hot-Encoding*-Array der Features erzeugt. Dies ist deshalb nötig, da im folgenden ein Modell für ein Multi-Label-Szenario trainiert werden soll: Ein Kursangebot kann den erwerb mehrerer Future Skills versprechen, etwa Softwareentwicklung **&** IT-Architektur. Das One-Hot-Encoding wird benötigt, um die Labels der Features in eine Form zu bringen, die das Classifier Modell von SetFit korrekt verarbeiten kann. Es stellt sicher, dass alle Klassen gleichwertig behandelt werden, und ermöglicht die Berechnung von Ähnlichkeiten verschiedener Kursinhalte.

Um den *One-Hot-Encoding*-Array zu erzeugen definieren wir `encode_labels(record)`: Diese Funktion geht durch jede Zeile eines Data Frames und erstellt eine neue Spalte namens `labels`. In dieser Spalte werden die Werte aus den oben definierten "features" gesammelt und in eine Liste gepackt. Diese Liste enthält Werte der Features.

Beispiel: Mit Blick auf die Features 'Data Analytics & KI', 'Softwareentwicklung' und 'Nutzerzentriertes Design', könnte ein Zeilenvektor des Arrays eines Kurses der sich mit Data Analytics & KI / Softwareentwicklung beschäftigt so aussehen: `[1, 1, 0]`

In [6]:
# Generiere Funktion für One-Hot-Encoding für jede Zeile (record)
def encode_labels(record):
    return {"labels": [record[feature] for feature in features]}

# Wende Funktion auf train_data an
dataset = dataset.map(encode_labels)

# Um die Daten übersichtlich im Viewer zu betrachten: Umwandlung und Pandas Data Frame
pd_dataset = dataset.to_pandas()

c:\Users\Hueck\AppData\Local\pypoetry\Cache\virtualenvs\future-skill-classification--ys6-NbF-py3.12\Lib\site-packages\datasets\utils\_dill.py:385: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Map: 100%|██████████| 1577/1577 [00:00<00:00, 4527.41 examples/s]


Wir kopieren den das `dataset`-Objekt in ein neues Objekt namens `train_data` und ziehen davon noch mals eine Kopie mit Pandas, um die Daten ein letztes Mal im Viewer zu prüfen.

In [7]:
train_dataset = dataset

# check data with pandas
pd_train_dataset = train_dataset.to_pandas() 

## Modelltraining

### Vorbereitung des Modells

Es wird das vortrainierte sentence-transformers-Modell `paraphrase-multilingual-MiniLM-L12-v2` aus der Hugging Face Bibliothek geladen. Durch die Angabe der Strategie `one-vs-rest` wird das Modell für eine Multi-Label-Klassifikation konfiguriert, bei der jede Klasse eines Kurses einzeln vorhergesagt wird. Auf diese Weise wird es möglich, mehrere Skills pro Kurs zu identifizieren. 

In [8]:
# Laden des Modells
 
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", 
    multi_target_strategy="multi-output",
    labels=['Data Analytics & KI', 'Softwareentwicklung', 'Nutzerzentriertes Design', 'IT-Architektur', 'Hardware/Robotikentwicklung', 'Quantencomputing'],
)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.




### Auswahl GPU/CPU

Es wird zunächst geprüft, ob CUDA (und damit eine GPU) verfügbar ist, indem `torch.cuda.is_available()` aufgerufen wird. Wenn CUDA verfügbar ist, wird die GPU als Rechengerät (`device`) ausgewählt, und der Name der GPU mit `torch.cuda.get_device_name(0)` ausgegeben. Falls CUDA nicht verfügbar ist, wird stattdessen die CPU als Rechengerät festgelegt. Anschließend wird das Modell mit `model.to(device)` entweder auf die GPU oder die CPU verschoben, abhängig von der zuvor getroffenen Geräteauswahl, um die Berechnungen entsprechend durchzuführen.

In [9]:
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
training_device

device(type='cuda')

In [10]:
import torch

# Prüfen, ob CUDA (GPU) verfügbar ist
print("CUDA verfügbar:", torch.cuda.is_available())

# Wenn CUDA verfügbar ist, die GPU verwenden
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("Verwende CPU")

# model auf GPU verschieben

    # Sicherstellen, dass das Modell auf der GPU ist, wenn verfügbar
model = model.to(device)


CUDA verfügbar: True
GPU: NVIDIA GeForce RTX 4060 Ti


Für den Fall dass eine GPU vorhanden ist: `PyTorch` wird verwendet, um die GPU für das Training von Modellen zu verwenden. `PyTorch` nutzt einen Speicher-Caching-Mechanismus, um GPU-Speicher effizient zu verwalten. Wenn Tensoren gelöscht oder nicht mehr benötigt werden, bleibt der Speicher oft noch im Cache, damit zukünftige Speicheranforderungen schneller bedient werden können. Der Befehl `torch.cuda.empty_cache()` leert im folgenden den Cache und gibt den belegten Speicher an das GPU-System zurück, ohne jedoch den aktuell in Verwendung befindlichen Speicher zu beeinflussen. Wird der Cache nicht regelmäßiig geleert, kann das einen ernormen Einfluss auf die Bearbeitungszeit des Trainings eines Classifiers haben.

In [11]:
import torch
torch.cuda.empty_cache()

Der `SetFitTrainer` wird instanziiert, um das Modell auf den Trainings- und Evaluierungsdatensätzen zu trainieren. Eine Bewertung des Modells wird erst zu einem späteren Zeitpunkt möglich, wenn Testdaten vorliegen.

1. Das vortrainierte `model` (siehe oben) wird als Grundlage für das Training verwendet, das auf dem Few-Shot Learning-Ansatz basiert. `paraphrase-mpnet-base-v2` ist ein vortrainiertes Modell aus der Sentence Transformers-Familie, das speziell für die Aufgabe der semantischen Textähnlichkeit und Paraphrasenerkennung entwickelt wurde. 
2. Der `train_dataset` enthält die Trainingsdaten, und der `eval_dataset` wird zu einem späteren Zeitpunkt genutzt, um die Klassifikations-Leistung des Modells während des Trainings zu bewerten.
3. Die Verlustfunktion (`loss_class`) wird als *Cosine Similarity Loss* festgelegt. Die *Cosine Similarity Loss* ist eine Verlustfunktion, die darauf basiert, wie ähnlich sich zwei Vektoren sind. In diesem Fall werden die Ausgabevektoren des Modells und die Zielvektoren (also die "Labels") mithilfe der Kosinus-Ähnlichkeit verglichen. Je höher die Ähnlichkeit also, desto besser das Modell. 
4. Die `column_mapping` gibt an, welche Spalten in den Datensätzen die Texte (`sentence` → `text`) und die zugehörigen Labels (`labels` → `label`) enthalten, sodass das Modell die Daten korrekt interpretieren kann.


In [12]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset, 
    #eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    num_iterations=40, 
    num_epochs=2,
    column_mapping={"sentence": "text", "labels": "label"}
)

C:\Users\Hueck\AppData\Local\Temp\ipykernel_18676\2679062307.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Map: 100%|██████████| 1577/1577 [00:00<00:00, 7282.30 examples/s]


Nun kann das Training beginnen: 

In [13]:
trainer.train()

***** Running training *****
  Num unique pairs = 126160
  Batch size = 16
  Num epochs = 2
  0%|          | 1/15770 [00:00<2:02:32,  2.14it/s]

{'embedding_loss': 0.1571, 'grad_norm': 1.779844045639038, 'learning_rate': 1.2682308180088778e-08, 'epoch': 0.0}


  0%|          | 51/15770 [00:06<31:34,  8.30it/s] 

{'embedding_loss': 0.1986, 'grad_norm': 2.48179292678833, 'learning_rate': 6.341154090044388e-07, 'epoch': 0.01}


  1%|          | 101/15770 [00:12<30:58,  8.43it/s]

...

{'embedding_loss': 0.0065, 'grad_norm': 0.28141674399375916, 'learning_rate': 9.864017473402382e-08, 'epoch': 1.99}


100%|█████████▉| 15751/15770 [31:43<00:02,  8.81it/s]

{'embedding_loss': 0.0048, 'grad_norm': 0.4094780683517456, 'learning_rate': 2.818290706686395e-08, 'epoch': 2.0}


100%|██████████| 15770/15770 [31:47<00:00,  8.27it/s]


{'train_runtime': 1907.3028, 'train_samples_per_second': 132.292, 'train_steps_per_second': 8.268, 'train_loss': 0.010429202906017298, 'epoch': 2.0}


In [14]:
# Speichern des Modells auf deinem PC
model.save_pretrained("models")  # Ersetze den Pfad durch deinen gewünschten Speicherort

In [15]:
# # Modell evaluieren (Nur bei vorhandensein von Testdaten)
# metrics = trainer.evaluate()
# metrics


In [ ]:
trainer.push_to_hub("Chernoffface/fs-setfit-multilable-model")

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("Chernoffface/fs-setfit-multilable-model")

In [ ]:
preds = model(
    [
        "Automation Projects, Automation Methods, Development Methods for Automation Systems, Automation with Qualitative Models, Safety and Reliability of Automation Systems",
        "Inhalt: I) Grundlagen - Motivation - Grundelemente der Quantenmechanik - Erinnerung II) Streuung in der Quantenmechanik - Formulierung des Streuproblems - Partialwellenentwicklung und Streuobservablen - Näherungsmethoden III) Quantenmechanisches Vielteilchenproblem - Unterscheidbare vs. identische Teilchen - Symmetrische und antisymmetrische Vielteilchenzustände - Zweite Quantisierung - Exakte Lösungen und Näherungsmethoden - Hartree-Fock & Gross-Pitaevskii Näherung IV) Relativistische Quantenmechanik - Erinnerung an die spez.",
        "Die Themen umfassen die Architektur moderner Spielesysteme, sowohl Hard- als auch Software, das Zeitmanagement im Millisekundenbereich, das Laden und Komprimieren von Assets sowie das physikalisch basierte Echtzeit-Rendering und Animationen. Weitere Schwerpunkte sind der Umgang mit großen Spielwelten, die Audiomodellierung und -mischung, physikbasierte Simulationen mittels Constraints sowie der Einsatz von künstlicher Intelligenz in Spielen. Zudem werden die Netzwerkkommunikation in Multiplayer-Spielen, die prozedurale Erstellung von Inhalten, die Integration von Skriptsprachen und die Optimierung sowie Parallelisierung von CPU- und GPU-Code behandelt. Die Übungen kombinieren dabei theoretische Grundlagen mit praktischen Anwendungen.",
        "Im Mittelpunkt des Seminars steht die neue Renaissance-Poesie, die aus Italien kommend die traditionelle spanische Lyrik zurückdrängt. Dichtungsformen und Inhalte werden beschrieben und die Wirkung auf die Literatur des siglo de oro festgehalten. Textausgaben: Juan Boscán, Obra completa, ed. Carlos Clavería, Madrid: Cátedra,  ISBN 978-84-376-1685-8 Garcilaso de la Vega, Poesia castellana completa, Madrid: Cátedra ISBN 978-84-376-0067-3 Die Texte sollen angeschafft werden. Weitere Texte werden gegebenenfalls vervielfältigt."
    ]
)
[[f for f, p in zip(features, ps) if p] for ps in preds]